In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans, MiniBatchKMeans, Birch, OPTICS
import tensorflow as tf
import re

In [3]:
df = pd.read_excel('Dataset+Harga.xlsx', sheet_name = 'Dataset (Gizi + Harga)')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592 entries, 0 to 591
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   makanan          592 non-null    object 
 1   jenis            592 non-null    object 
 2   air_(g)          592 non-null    float64
 3   energi_(kal)     592 non-null    int64  
 4   protein_(g)      592 non-null    float64
 5   lemak_(g)        592 non-null    object 
 6   karbohidrat_(g)  592 non-null    float64
 7   serat_(g)        592 non-null    object 
 8   Harga (Rp)       592 non-null    int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 41.8+ KB


In [5]:
df.nunique()

makanan            592
jenis                6
air_(g)            395
energi_(kal)       334
protein_(g)        219
lemak_(g)          213
karbohidrat_(g)    376
serat_(g)           86
Harga (Rp)          74
dtype: int64

In [6]:
#Get info from the dataframe
variables = []
dtypes = []
count = []
unique = []
missing = []

for item in df.columns:
    variables.append(item)
    dtypes.append(df[item].dtype)
    count.append(len(df[item]))
    unique.append(len(df[item].unique()))
    missing.append(df[item].isna().sum())

output = pd.DataFrame({
    'variable': variables,
    'dtype': dtypes,
    'count': count,
    'unique': unique,
    'missing': missing,

})

output

,variable,dtype,count,unique,missing
0,makanan,object,592,592,0
1,jenis,object,592,6,0
2,air_(g),float64,592,395,0
3,energi_(kal),int64,592,334,0
4,protein_(g),float64,592,219,0
5,lemak_(g),object,592,213,0
6,karbohidrat_(g),float64,592,376,0
7,serat_(g),object,592,86,0
8,Harga (Rp),int64,592,74,0


In [7]:
df['jenis'].value_counts()

lauk                184
makanan_tambahan    181
buah                113
sayur                62
makanan_pokok        46
susu                  6
Name: jenis, dtype: int64

In [8]:
df.head()

,makanan,jenis,air_(g),energi_(kal),protein_(g),lemak_(g),karbohidrat_(g),serat_(g),Harga (Rp)
0,Nasi Putih,makanan_pokok,56.7,180,3.0,0.3,39.8,0.2,4000
1,Nasi tim,makanan_pokok,71.0,120,2.4,0.4,26.0,0.5,15000
2,Tapai,makanan_pokok,75.5,99,1.7,0.3,22.4,0,40000
3,Nasi Merah,makanan_pokok,64.0,149,2.8,0.4,32.5,0.3,8000
4,Bihun Jagung,makanan_pokok,11.3,354,0.5,0.3,87.4,3,7000


In [9]:
df.tail()

,makanan,jenis,air_(g),energi_(kal),protein_(g),lemak_(g),karbohidrat_(g),serat_(g),Harga (Rp)
587,Es krim,makanan_tambahan,62.1,210,4.0,12.5,20.6,0,5000
588,Hangop,makanan_tambahan,84.0,65,10.0,1,3.5,0,30000
589,Keju,makanan_tambahan,38.5,326,22.8,20.3,13.1,0,15000
590,Kwark (Quark),makanan_tambahan,59.0,242,19.0,17,3.0,0,30000
591,"Yoghurt, segar",susu,88.0,52,3.3,2.5,4.0,0,15000


In [10]:
df.sort_values(by='energi_(kal)', ascending=False)

,makanan,jenis,air_(g),energi_(kal),protein_(g),lemak_(g),karbohidrat_(g),serat_(g),Harga (Rp)
165,"Kacang mete/biji jambu monyet, goreng",makanan_tambahan,1.0,629,20.4,56.3,19.8,1.1,85000
334,"Kelapa hutan, kering",buah,6.0,623,2.0,"52, 1",36.5,15.3,10000
540,"Dendeng mujahir, goreng, masakan",lauk,6.5,598,74.3,26.9,9.2,0,5000
192,Keripik oncom,makanan_tambahan,3.0,598,8.3,43.8,42.6,3.3,17500
60,Kue kelapa,makanan_tambahan,3.3,591,5.6,42.1,47.5,0,12000
...,...,...,...,...,...,...,...,...,...
244,"Selada, rebus",sayur,93.5,19,2.7,0.2,2.9,0.7,4000
243,"Paria putih, kukus",sayur,95.0,16,0.8,0.2,3.4,1.2,9000
268,Lema/ Rebung asam,sayur,96.3,15,1.0,0.4,1.9,0,15000
551,"Jukku pallu kaloa, masakan",lauk,69.4,15,15.2,4.3,8.8,0,15000


In [11]:
df.sort_values(by='protein_(g)', ascending=False)

,makanan,jenis,air_(g),energi_(kal),protein_(g),lemak_(g),karbohidrat_(g),serat_(g),Harga (Rp)
540,"Dendeng mujahir, goreng, masakan",lauk,6.5,598,74.3,26.9,9.2,0,5000
518,"Ikan kayu, kering",lauk,23.1,302,70.7,1.9,0.4,0,20000
524,"Ikan mujahir, dendeng, mentah",lauk,15.2,582,68.3,15.2,37.2,0,15000
534,"Ikan sale lais, mentah",lauk,12.4,415,61.3,17.6,2.9,0,15000
514,"Ikan Gabus, kering",lauk,24.0,268,58.0,4,0.0,0,25000
...,...,...,...,...,...,...,...,...,...
146,Mie Sagu,makanan_tambahan,52.8,190,0.2,0.4,46.5,1.4,8000
144,Mi golosor,makanan_pokok,82.9,71,0.2,0.8,15.9,0.1,7000
134,Keripik ubi,makanan_tambahan,6.0,486,0.2,23.9,67.6,14.3,15000
118,Bubur sagu,makanan_pokok,62.5,167,0.2,4,33.0,0,54000


In [12]:
#Find correlation between each column
df.corr()

<ipython-input-12-4d5f2adea923>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,air_(g),energi_(kal),protein_(g),karbohidrat_(g),Harga (Rp)
air_(g),1.000000,-0.949499,-0.418951,-0.693201,-0.017793
energi_(kal),-0.949499,1.000000,0.434585,0.563150,0.048556
protein_(g),-0.418951,0.434585,1.000000,-0.252250,0.084485
karbohidrat_(g),-0.693201,0.563150,-0.252250,1.000000,-0.037271
Harga (Rp),-0.017793,0.048556,0.084485,-0.037271,1.000000


In [13]:
data = df[['makanan', 'jenis', 'energi_(kal)', 'Harga (Rp)']]

In [14]:
data

,makanan,jenis,energi_(kal),Harga (Rp)
0,Nasi Putih,makanan_pokok,180,4000
1,Nasi tim,makanan_pokok,120,15000
2,Tapai,makanan_pokok,99,40000
3,Nasi Merah,makanan_pokok,149,8000
4,Bihun Jagung,makanan_pokok,354,7000
...,...,...,...,...
587,Es krim,makanan_tambahan,210,5000
588,Hangop,makanan_tambahan,65,30000
589,Keju,makanan_tambahan,326,15000
590,Kwark (Quark),makanan_tambahan,242,30000


In [15]:
# le = LabelEncoder()
# data['jenis'] = le.fit_transform(data['jenis'])

In [16]:
# # Buat dictionary untuk mapping
# mapping = {'susu': 1, 'buah': 2, 'sayur': 3, 'makanan_tambahan': 4, 'lauk':5, 'makanan_pokok':6}

# # Ganti nilai kategorikal dengan angka yang ditentukan
# data['jenis'] = data['jenis'].replace(mapping)


In [17]:
data

,makanan,jenis,energi_(kal),Harga (Rp)
0,Nasi Putih,makanan_pokok,180,4000
1,Nasi tim,makanan_pokok,120,15000
2,Tapai,makanan_pokok,99,40000
3,Nasi Merah,makanan_pokok,149,8000
4,Bihun Jagung,makanan_pokok,354,7000
...,...,...,...,...
587,Es krim,makanan_tambahan,210,5000
588,Hangop,makanan_tambahan,65,30000
589,Keju,makanan_tambahan,326,15000
590,Kwark (Quark),makanan_tambahan,242,30000


In [18]:
# Pengguna mengisi informasi pribadi
nama = input("Masukkan nama Anda: ")
jenis_kelamin = input("Masukkan jenis kelamin (L/P): ")
tinggi_badan = float(input("Masukkan tinggi badan Anda (cm): "))
berat_badan = float(input("Masukkan berat badan Anda (kg): "))
alergi = input("Apakah Anda memiliki alergi makanan? (Y/N): ")


Masukkan nama Anda: Dodo
Masukkan jenis kelamin (L/P): L
Masukkan tinggi badan Anda (cm): 170
Masukkan berat badan Anda (kg): 90
Apakah Anda memiliki alergi makanan? (Y/N): Y


In [19]:

# Pengguna menginput budget makanan dan berat badan tujuan
budget_makan = float(input("Masukkan budget makanan Anda: "))
berat_badan_tujuan = float(input("Masukkan berat badan tujuan Anda (kg): "))


Masukkan budget makanan Anda: 30000
Masukkan berat badan tujuan Anda (kg): 60


In [20]:
#Perhitungan Kebutuhan Kalori Harian
kebutuhan_kalori = 15.3 * berat_badan + 679
# Menyesuaikan kebutuhan kalori dengan budget yang telah diinput
kebutuhan_kalori = kebutuhan_kalori * (berat_badan_tujuan / berat_badan)


In [21]:
def filter_alergi(data, alergi_makanan):
    for alergi in alergi_makanan:
        regex = re.compile(r'\b' + re.escape(alergi) + r'\b', flags=re.IGNORECASE)
        data = [item for item in data if not regex.search(item)]
    return data

if alergi.upper() == 'Y':
    jumlah_alergi = int(input("Masukkan jumlah alergi makanan: "))
    alergi_makanan = []
    for i in range(jumlah_alergi):
        alergi = input(f"Masukkan jenis makanan alergi ke-{i+1}: ")
        alergi_makanan.append(alergi)
    data = filter_alergi(data, alergi_makanan)


Masukkan jumlah alergi makanan: 1
Masukkan jenis makanan alergi ke-1: Tikus


In [22]:
X = df[['energi_(kal)', 'Harga (Rp)']]

# Scaling data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(X)

optics = OPTICS()

num_clusters = 7
gmm = GaussianMixture(n_components=num_clusters)
labels = gmm.fit_predict(scaled_data)

In [24]:
# Model untuk rekomendasi makanan
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(num_clusters, activation='softmax')
])

model.compile(tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(scaled_data, labels, epochs=1000, batch_size=128)

Epoch 1/1000
5/5 [==============================] - 1s 8ms/step - loss: 1.9404 - accuracy: 0.2770
Epoch 2/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.9389 - accuracy: 0.2770
Epoch 3/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.9376 - accuracy: 0.2770
Epoch 4/1000
5/5 [==============================] - 0s 5ms/step - loss: 1.9362 - accuracy: 0.2770
Epoch 5/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.9346 - accuracy: 0.2770
Epoch 6/1000
5/5 [==============================] - 0s 5ms/step - loss: 1.9331 - accuracy: 0.2770
Epoch 7/1000
5/5 [==============================] - 0s 4ms/step - loss: 1.9316 - accuracy: 0.2770
Epoch 8/1000
5/5 [==============================] - 0s 4ms/step - loss: 1.9300 - accuracy: 0.2770
Epoch 9/1000
5/5 [==============================] - 0s 5ms/step - loss: 1.9284 - accuracy: 0.2770
Epoch 10/1000
5/5 [==============================] - 0s 5ms/step - loss: 1.9269 - accuracy: 0.2770
Epoch 11/1000
5/5 [

In [25]:
# Saving the model in .h5 format
model.save('recommendation_model_gmm.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
# Convert the model to TensorFlow Lite format (.tflite)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [27]:
# Save the TensorFlow Lite model to a file
with open('recommendation_model_gmm.tflite', 'wb') as f:
    f.write(tflite_model)

In [28]:
# Rekomendasi makanan berdasarkan berat badan, berat badan tujuan, dan budget
cluster = gmm.predict([[kebutuhan_kalori, budget_makan]])[0]
cluster_data = df[gmm.predict(scaled_data) == cluster]
recommended_foods = cluster_data[['makanan', 'jenis', 'energi_(kal)',
                                  'air_(g)', 'protein_(g)', 'lemak_(g)',
                                  'karbohidrat_(g)', 'serat_(g)', 'Harga (Rp)']].values.tolist()


In [ ]:
# Output rekomendasi makanan untuk pengguna
# print("Rekomendasi makanan:")
# for food in recommended_foods:
#     print(f"Makanan: {food[0]}")
#     print(f"Jenis: {food[1]}")
#     print(f"Energi (kal): {food[2]}")
#     print(f"Harga: {food[3]}")
#     print("\n")

In [29]:
# Mendefinisikan setiap jenis makanan
jenis_makanan = ['makanan pokok', 'makanan tambahan', 'lauk', 'susu', 'sayur', 'buah']
recommended_foods = []

basic_recommendations = {
    'makanan pokok': 'Nasi Merah',
    'makanan tambahan': 'Keripik Tempe',
    'lauk': 'Tahu Telur',
    'sayur': 'Bayam Kukus',
    'buah': 'Semangka, segar',
    'susu': 'Susu sapi, segar'
}

# Memilih satu makanan dari setiap jenis
for jenis in jenis_makanan:
    jenis_data = df[df['jenis'] == jenis]['makanan'].tolist()

    # Memilih 1 sampel acak dari setiap jenis makanan (jika tersedia lebih dari 1)
    if len(jenis_data) > 0:
        recommended_food = np.random.choice(jenis_data)
        recommended_foods.append(recommended_food)
    else:
        recommended_foods.append(basic_recommendations[jenis])

# Output rekomendasi makanan
print("Rekomendasi makanan:")
for i, jenis in enumerate(jenis_makanan):
    print(f"{jenis.capitalize()}: {recommended_foods[i]}")


Rekomendasi makanan:
Makanan pokok: Nasi Merah
Makanan tambahan: Keripik Tempe
Lauk: Ikan saluang, kering, mentah 
Susu: Yoghurt, segar 
Sayur: Asinan Bogor, sayuran 
Buah: Pisang ketip, segar 


In [30]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="recommendation_model_gmm.tflite")
interpreter.allocate_tensors()

In [31]:
# Mendefinisikan input_data untuk inferensi
input_data = np.array([[kebutuhan_kalori, budget_makan]], dtype=np.float32)
input_data = input_data.reshape((1, 2))

# Set input tensor dan invoke interpreter
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]
interpreter.set_tensor(input_index, input_data)
interpreter.invoke()

In [32]:
# Mendapatkan hasil prediksi dari model
predictions = interpreter.get_tensor(output_index)

# Output hasil prediksi
print("Hasil prediksi:")
print(predictions)

Hasil prediksi:
[[0. 0. 0. 0. 0. 1. 0.]]


In [ ]:
# # Mendefinisikan rekomendasi makanan berdasarkan klaster
# klaster_rekomendasi = {
#     0: 'Nasi Merah',
#     1: 'Keripik Tempe',
#     2: 'Tahu Telur',
#     3: 'Bayam Kukus',
#     4: 'Semangka, segar',
#     5: 'Susu sapi, segar'
# }

# # Klaster terpilih berdasarkan hasil prediksi
# klaster_terpilih = np.argmax(predictions)

# # Menghubungkan jenis makanan dengan klaster rekomendasi
# jenis_to_klaster = {
#     'makanan pokok': 0,
#     'makanan tambahan': 1,
#     'lauk': 2,
#     'sayur': 3,
#     'buah': 4,
#     'susu': 5
# }

# # Membuat rekomendasi berdasarkan klaster yang dipilih
# recommended_foods = []
# for jenis in jenis_makanan:
#     klaster = jenis_to_klaster[jenis]
#     recommended_foods.append(klaster_rekomendasi[klaster])

# # Output rekomendasi makanan
# print("Rekomendasi makanan:")
# for i, jenis in enumerate(jenis_makanan):
#     print(f"{jenis.capitalize()}: {recommended_foods[i]}")
